# Compare VCF data

In [ ]:
!venv/bin/pip install intervaltree

In [124]:
!samtools depth WES_ICC/HCC1239_1_3.dedupped.realigned.recal.bam > reads.sort.coverage

In [137]:
from intervaltree import Interval, IntervalTree
import vcf

In [126]:
test_coverage = """chr1    10009   1
chr1    10010   2
chr1    10011   5
chr1    10012   4
chr1    10013   5
chr1    10014   5
chr1    10015   5
chr2    10016   5
chr2    10017   5
chr2    10018   5
chr2    10019   2
chr2    10020   5"""

In [127]:
def coverage_by_chromosome(iterable, threshold):
    coverage_intervals = []
    total_coverage = {}
    last_chromosome = None
    last_streak = False
    try:
        while True:
            elem = next(iterable)
            chromosome, coord, coverage = elem.split()
            if not last_chromosome:
                last_chromosome = chromosome
            coord = int(coord)
            coverage = int(coverage)
            streak = coverage >= threshold
            if chromosome != last_chromosome:
                if streak:
                    current_interval.append(coord - 1)
                    coverage_intervals.append(current_interval)
                total_coverage[last_chromosome] = coverage_intervals
                coverage_intervals = []
                last_streak = False

            if not last_streak and streak:
                current_interval = [coord]
            if last_streak and not streak:
                current_interval.append(coord - 1)
                coverage_intervals.append(current_interval)
                current_interval = []
            last_streak = streak
            last_chromosome = chromosome
    except StopIteration:
        if streak:
            current_interval.append(coord)
            coverage_intervals.append(current_interval)
        total_coverage[last_chromosome] = coverage_intervals
    return total_coverage

In [128]:
iterable = (i for i in test_coverage.split("\n"))
threshold = 5
result = coverage_by_chromosome(iterable, threshold)
assert result == {'chr1': [[10011, 10011], [10013, 10015]], 'chr2': [[10016, 10018], [10020, 10020]]}

In [129]:
def make_intervaltrees(chromomsome_intervals):
    intervaltrees = {}
    for c, intervals in chromomsome_intervals.items():
        it = intervaltrees.setdefault(c, IntervalTree())
        for interval in intervals:
            it[interval[0]: interval[1] + 1] = None
    return intervaltrees

In [130]:
test_coverage2 = """chr1    110703   5
chr1    110704   2
chr1    110705   5
chr1    110706   4
chr1    110707   5
chr1    110708   5
chr1    110709   5
chr1    110710   5
chr1    110711   5
chr1    110712   5
chr1    110713   2
chr1    110714   5"""

In [132]:
test_intervaltrees2 = make_intervaltrees(coverage_by_chromosome((i for i in test_coverage2.split("\n")), 5))

In [133]:
test_intervaltrees2

{'chr1': IntervalTree([Interval(110703, 110704), Interval(110705, 110706), Interval(110707, 110713), Interval(110714, 110715)])}

In [134]:
with open("reads.sort.coverage") as f:
    coverage_intervals = coverage_by_chromosome(f, 5)

In [135]:
coverage_intervals = make_intervaltrees(coverage_intervals)

In [141]:
#coverage_intervals["chr1"][861347]

In [142]:
def filter_vcf(filename, coverage_intervals, limit=-1, chrom_limit="", phased=False, min_qual=50):
    write_to = ".".join(filename.split(".")[:-1]) + ".filtered.vcf"
    reader = vcf.Reader(filename=filename)
    with open(write_to, "w") as f:
        writer = vcf.Writer(f, reader)
        for i, record in enumerate(reader):
            #print(record.samples[0])
            #print(record.CHROM, record.POS)
            if record.CHROM == chrom_limit:
                break
            if i == limit:
                break
            if record.samples[0].data.GQ < min_qual:
                continue
            if coverage_intervals[record.CHROM][record.POS]:
                writer.write_record(record)

In [144]:
filter_vcf("reverse_jewish_son.vcf.gz", coverage_intervals)